In [ ]:
from glob import glob
from tqdm.notebook import tqdm
import yaml
import pickle
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import lightgbm as lgb
import xgboost as xgb
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, SequentialSampler
import ubiquant

In [ ]:
df_train = pd.read_pickle('../input/ubiquant-market-prediction-eda/train.pkl')
print(f'Training Set Shape: {df_train.shape} - Memory Usage: {df_train.memory_usage().sum() / 1024 ** 2:.2f} MB')

## Linear Model

In [ ]:
def load_linear_model(path):
    
    config = yaml.load(open(f'{path}/config.yaml', 'r'), Loader=yaml.FullLoader)
    with open(f'{path}/model', 'rb') as f:
        model = pickle.load(f)

    return config, model


#linear_model_config, linear_model = load_linear_model('../input/ubiquant-market-prediction-dataset/linear_model/no_split')
#linear_model

## LightGBM

In [ ]:
def load_lgb_model(path):
    
    """
    Load LightGBM models and config file from the given path

    Parameters
    ----------
    path (str): Path of the model directory
    
    Returns
    -------
    config (dict): Dictionary of model configurations
    models (list): List of trained LightGBM models
    """
    
    config = yaml.load(open(f'{path}/config.yaml', 'r'), Loader=yaml.FullLoader)
    models = [lgb.Booster(model_file=model_path) for model_path in glob(f'{path}/model*.txt')]

    return config, models


lgb_config, lgb_models = load_lgb_model('../input/ubiquant-market-prediction-dataset/lightgbm_regression/no_split')

## XGBoost

In [ ]:
def load_xgb_model(path):
    
    """
    Load XGBoost models and config file from the given path

    Parameters
    ----------
    path (str): Path of the model directory
    
    Returns
    -------
    config (dict): Dictionary of model configurations
    models (list): List of trained XGBoost models
    """
    
    config = yaml.load(open(f'{path}/config.yaml', 'r'), Loader=yaml.FullLoader)
    models = [xgb.Booster(model_file=model_path) for model_path in glob(f'{path}/model*.json')]

    return config, models


xgb_config, xgb_models = load_xgb_model('../input/ubiquant-market-prediction-dataset/xgboost_regression/no_split')

## PyTorch Datasets

In [ ]:
class TabularDataset(Dataset):

    def __init__(self, features, labels=None):

        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):

        """
        Get the idxth element in the dataset

        Parameters
        ----------
        idx (int): Index of the sample (0 <= idx < len(self.features))

        Returns
        -------
        features [torch.FloatTensor of shape (1, n_features)]: Features
        label [torch.FloatTensor of shape (1)]: Label
        """

        features = self.features[idx]
        features = torch.tensor(features, dtype=torch.float)

        if self.labels is not None:
            label = self.labels[idx]
            label = torch.tensor(label, dtype=torch.float)
            return features, label
        else:
            return features


## Multilayer Perceptron

In [ ]:
def init_weights(module,
                 linear_weight_init_type, linear_weight_init_args, linear_bias_init_type, linear_bias_init_args,
                 batch_normalization_weight_init_type=None, batch_normalization_weight_init_args=None,
                 batch_normalization_bias_init_type=None, batch_normalization_bias_init_args=None):

    """
    Initialize weights and biases of given layers with specified configurations

    Parameters
    ----------
    module (torch.nn.Module): Layer
    linear_weight_init_type (str): Weight initialization method of the linear layer
    linear_weight_init_args (dict): Weight initialization arguments of the linear layer
    linear_bias_init_type (str): Bias initialization method of the linear layer
    linear_bias_init_args (dict): Bias initialization arguments of the linear layer
    batch_normalization_weight_init_type (str): Weight initialization method of the batch normalization layer
    batch_normalization_weight_init_args (dict): Weight initialization arguments of the batch normalization layer
    batch_normalization_bias_init_type (str): Bias initialization method of the batch normalization layer
    batch_normalization_bias_init_args (dict): Bias initialization arguments of the batch normalization layer
    """

    if isinstance(module, nn.Linear):
        # Initialize weights of linear layer
        if linear_weight_init_type == 'uniform':
            nn.init.uniform_(
                module.weight,
                a=linear_weight_init_args['a'],
                b=linear_weight_init_args['b']
            )
        elif linear_weight_init_type == 'normal':
            nn.init.normal_(
                module.weight,
                mean=linear_weight_init_args['mean'],
                std=linear_weight_init_args['std']
            )
        elif linear_weight_init_type == 'xavier_uniform':
            nn.init.xavier_uniform_(
                module.weight,
                gain=nn.init.calculate_gain(
                    nonlinearity=linear_weight_init_args['nonlinearity'],
                    param=linear_weight_init_args['nonlinearity_param']
                )
            )
        elif linear_weight_init_type == 'xavier_normal':
            nn.init.xavier_normal_(
                module.weight,
                gain=nn.init.calculate_gain(
                    nonlinearity=linear_weight_init_args['nonlinearity'],
                    param=linear_weight_init_args['nonlinearity_param']
                )
            )
        elif linear_weight_init_type == 'kaiming_uniform':
            nn.init.kaiming_uniform_(
                module.weight,
                a=linear_weight_init_args['nonlinearity_param'],
                mode=linear_weight_init_args['mode'],
                nonlinearity=linear_weight_init_args['nonlinearity']
            )
        elif linear_weight_init_type == 'kaiming_normal':
            nn.init.kaiming_normal_(
                module.weight,
                a=linear_weight_init_args['nonlinearity_param'],
                mode=linear_weight_init_args['mode'],
                nonlinearity=linear_weight_init_args['nonlinearity']
            )
        elif linear_weight_init_type == 'orthogonal':
            nn.init.orthogonal_(
                module.weight,
                gain=nn.init.calculate_gain(
                    nonlinearity=linear_weight_init_args['nonlinearity'],
                    param=linear_weight_init_args['nonlinearity_param']
                )
            )
        elif linear_weight_init_type == 'sparse':
            nn.init.sparse_(
                module.weight,
                sparsity=linear_weight_init_args['sparsity'],
                std=linear_weight_init_args['std']
            )
        # Initialize biases of Linear layer
        if module.bias is not None:
            if linear_bias_init_type == 'uniform':
                nn.init.uniform_(
                    module.bias,
                    a=linear_bias_init_args['a'],
                    b=linear_bias_init_args['b']
                )
            elif linear_bias_init_type == 'normal':
                nn.init.normal_(
                    module.bias,
                    mean=linear_bias_init_args['mean'],
                    std=linear_bias_init_args['std']
                )

    elif isinstance(module, nn.BatchNorm1d):
        # Initialize weights of batch normalization layer
        if batch_normalization_weight_init_type is not None:
            if batch_normalization_weight_init_type == 'uniform':
                nn.init.uniform_(
                    module.weight,
                    a=batch_normalization_weight_init_args['a'],
                    b=batch_normalization_weight_init_args['b']
                )
            elif batch_normalization_weight_init_type == 'normal':
                nn.init.normal_(
                    module.weight,
                    mean=batch_normalization_weight_init_args['mean'],
                    std=batch_normalization_weight_init_args['std']
                )
            elif batch_normalization_weight_init_type == 'constant':
                nn.init.constant_(
                    module.weight,
                    val=batch_normalization_weight_init_args['val'],
                )
        # Initialize biases of batch normalization layer
        if batch_normalization_bias_init_type is not None:
            if batch_normalization_bias_init_type == 'uniform':
                nn.init.uniform_(
                    module.bias,
                    a=batch_normalization_bias_init_args['a'],
                    b=batch_normalization_bias_init_args['b']
                )
            elif batch_normalization_bias_init_type == 'normal':
                nn.init.normal_(
                    module.bias,
                    mean=batch_normalization_bias_init_args['mean'],
                    std=batch_normalization_bias_init_args['std']
                )
            elif batch_normalization_bias_init_type == 'constant':
                nn.init.constant_(
                    module.bias,
                    val=batch_normalization_bias_init_args['val'],
                )


class DenseBlock(nn.Module):

    def __init__(self, input_dim, output_dim, batch_normalization, weight_normalization, dropout_probability, activation, activation_args, init_args):

        """
        Vanilla dense block (Linear -> Batch Normalization -> Dropout -> Activation)

        Parameters
        ----------
        input_dim (int): Number of input dimensions of the dense layer
        output_dim (int): Number of output dimensions of the dense layer
        batch_normalization (bool): Whether to add batch normalization or not
        weight_normalization (bool): Whether to add weight normalization or not
        dropout_probability (int): Probability of the dropout
        activation (str): Class name of the activation function
        activation_args (dict): Class arguments of the activation function
        init_args (dict): Weight and bias initialization arguments of the layers
        """

        super(DenseBlock, self).__init__()

        if weight_normalization:
            self.linear = nn.utils.weight_norm(nn.Linear(in_features=input_dim, out_features=output_dim, bias=True))
        else:
            self.linear = nn.Linear(in_features=input_dim, out_features=output_dim, bias=True)

        if batch_normalization:
            self.batch_normalization = nn.BatchNorm1d(num_features=output_dim)
        else:
            self.batch_normalization = nn.Identity()

        if dropout_probability > 0.0:
            self.dropout = nn.Dropout(p=dropout_probability)
        else:
            self.dropout = nn.Identity()

        if activation is not None and activation != 'Swish':
            self.activation = getattr(nn, activation)(**activation_args)
        elif activation == 'Swish':
            self.activation = Swish()
        else:
            self.activation = nn.Identity()

        if init_args is not None:
            init_weights(self.linear, **init_args)
            if batch_normalization:
                init_weights(self.batch_normalization, **init_args)

    def forward(self, x):

        x = self.linear(x)
        x = self.batch_normalization(x)
        x = self.dropout(x)
        x = self.activation(x)

        return x


class MultiLayerPerceptron(nn.Module):

    def __init__(self, input_dim, batch_normalization, weight_normalization, dropout_probability, activation, activation_args, init_args):

        """
        MLP (Multi Layer Perceptron) with multiple dense blocks

        Parameters
        ----------
        input_dim (int): Number of input dimensions of the first dense block
        batch_normalization (bool): Whether to add batch normalization to dense blocks or not
        weight_normalization (bool): Whether to add weight normalization to dense blocks or not
        dropout_probability (int): Probability of the dropout in dense blocks
        activation (str): Class name of the activation function in dense blocks
        activation_args (dict): Class arguments of the activation function in dense blocks
        init_args (dict): Weight and bias initialization arguments of the layers in dense blocks
        """

        super(MultiLayerPerceptron, self).__init__()

        self.dense_block1 = DenseBlock(
            input_dim=input_dim,
            output_dim=512,
            batch_normalization=batch_normalization,
            weight_normalization=weight_normalization,
            dropout_probability=dropout_probability,
            activation=activation,
            activation_args=activation_args,
            init_args=init_args
        )
        self.dense_block2 = DenseBlock(
            input_dim=512,
            output_dim=768,
            batch_normalization=batch_normalization,
            weight_normalization=weight_normalization,
            dropout_probability=dropout_probability,
            activation=activation,
            activation_args=activation_args,
            init_args=init_args
        )
        self.dense_block3 = DenseBlock(
            input_dim=768,
            output_dim=1024,
            batch_normalization=batch_normalization,
            weight_normalization=weight_normalization,
            dropout_probability=dropout_probability,
            activation=activation,
            activation_args=activation_args,
            init_args=init_args
        )
        self.dense_block4 = DenseBlock(
            input_dim=1024,
            output_dim=768,
            batch_normalization=batch_normalization,
            weight_normalization=weight_normalization,
            dropout_probability=dropout_probability,
            activation=activation,
            activation_args=activation_args,
            init_args=init_args
        )
        self.dense_block5 = DenseBlock(
            input_dim=768,
            output_dim=512,
            batch_normalization=batch_normalization,
            weight_normalization=weight_normalization,
            dropout_probability=dropout_probability,
            activation=activation,
            activation_args=activation_args,
            init_args=init_args
        )
        self.dense_block6 = DenseBlock(
            input_dim=512,
            output_dim=256,
            batch_normalization=batch_normalization,
            weight_normalization=weight_normalization,
            dropout_probability=dropout_probability,
            activation=activation,
            activation_args=activation_args,
            init_args=init_args
        )
        self.dense_block7 = DenseBlock(
            input_dim=256,
            output_dim=128,
            batch_normalization=batch_normalization,
            weight_normalization=weight_normalization,
            dropout_probability=dropout_probability,
            activation=activation,
            activation_args=activation_args,
            init_args=init_args
        )
        self.head = DenseBlock(
            input_dim=128,
            output_dim=1,
            batch_normalization=False,
            weight_normalization=False,
            dropout_probability=0.0,
            activation=None,
            activation_args=None,
            init_args=init_args
        )

    def forward(self, x):

        x = self.dense_block1(x)
        x = self.dense_block2(x)
        x = self.dense_block3(x)
        x = self.dense_block4(x)
        x = self.dense_block5(x)
        x = self.dense_block6(x)
        x = self.dense_block7(x)
        output = self.head(x)

        return output.view(-1)


In [ ]:
def load_mlp_model(path):
    
    """
    Load Multilayer Perceptron models and config file from the given path

    Parameters
    ----------
    path (str): Path of the model directory
    
    Returns
    -------
    config (dict): Dictionary of model configurations
    models (list): List of trained Multilayer Perceptron models
    """
    
    config = yaml.load(open(f'{path}/config.yaml', 'r'), Loader=yaml.FullLoader)
    models = []
    
    for model_path in glob(f'{path}/model*.pt'):
        
        model = MultiLayerPerceptron(
            input_dim=300,
            batch_normalization=True,
            weight_normalization=False,
            dropout_probability=0.2,
            activation='LeakyReLU',
            activation_args={
                'negative_slope': 0.1
            },
            init_args={
                'linear_weight_init_type': 'kaiming_normal',
                'linear_weight_init_args': {
                    'mode': 'fan_out',
                    'nonlinearity': 'leaky_relu',
                    'nonlinearity_param': 0.1
                },
                'linear_bias_init_type': 'normal',
                'linear_bias_init_args': {
                    'mean': 0.0,
                    'std': 1.0
                },
                'batch_normalization_weight_init_type': None,
                'batch_normalization_weight_init_args': None,
                'batch_normalization_bias_init_type': None,
                'batch_normalization_bias_init_args': None
            }

        )
        model.load_state_dict(torch.load(model_path))
        model.eval()
        model.to(torch.device('cuda'))
        models.append(model)

    return config, models


mlp_config, mlp_models = load_mlp_model('../input/ubiquant-market-prediction-dataset/mlp_regression/no_split')

## Post-processing

In [ ]:
def _pearson_correlation_coefficient(df, col_x, col_y):

    """
    Calculate Pearson correlation coefficient between two columns

    Parameters
    ----------
    df [pandas.DataFrame of shape (n_samples, 2)]: Training set with two columns
    col_x (str): Name of the first column
    col_y (str): Name of the second column
    """

    return df.corr()[col_x][col_y]


def mean_pearson_correlation_coefficient(df, col_x, col_y):

    """
    Calculate Pearson correlation coefficient between two columns for every time_id and average their results

    Parameters
    ----------
    df [pandas.DataFrame of shape (n_samples, 3)]: Training set with time_id and two columns
    col_x (str): Name of the first column
    col_y (str): Name of the second column
    """

    return np.mean(df[['time_id', col_x, col_y]].groupby('time_id').apply(_pearson_correlation_coefficient, col_x, col_y))


In [ ]:
df_lgb_regression_predictions = pd.read_csv('../input/ubiquant-market-prediction-dataset/lightgbm_regression/single_split/predictions.csv')
df_xgb_regression_predictions = pd.read_csv('../input/ubiquant-market-prediction-dataset/xgboost_regression/single_split/predictions.csv')
df_mlp_regression_predictions = pd.read_csv('../input/ubiquant-market-prediction-dataset/mlp_regression/single_split/predictions.csv')

df_train['lgb_regression_predictions'] = df_lgb_regression_predictions['predictions'].values
df_train['xgb_regression_predictions'] = df_xgb_regression_predictions['predictions'].values
df_train['mlp_regression_predictions'] = df_mlp_regression_predictions['predictions'].values

models_and_prediction_columns = {
    'LightGBM Regression': 'lgb_regression_predictions',
    'XGBoost Regression': 'xgb_regression_predictions',
    'Multilayer Perceptron': 'mlp_regression_predictions'
}

for model_name, prediction_column in models_and_prediction_columns.items():
    score = mean_pearson_correlation_coefficient(df_train, col_x='target', col_y=prediction_column)
    print(f'{model_name} - Validation Mean Pearson Correlation Coefficient: {score:.6f}')

In [ ]:
df_train['blend_predictions'] = (df_train['lgb_regression_predictions'] * 1) + (df_train['xgb_regression_predictions'] * 1) + (df_train['mlp_regression_predictions'] / 1000)
score = mean_pearson_correlation_coefficient(df_train, col_x='target', col_y='blend_predictions')
print(f'Blend - Validation Mean Pearson Correlation Coefficient: {score:.6f}')

## Inference

In [ ]:
env = ubiquant.make_env()
iter_test = env.iter_test()

for df_test, df_sample_submission in iter_test:
    
    '''# LightGBM Inference
    lgb_predictions = np.zeros(len(df_test))
    for lgb_model in lgb_models:
        lgb_predictions += (lgb_model.predict(df_test.loc[:, lgb_config['features']]) / len(lgb_models))
    
    # XGBoost Inference
    xgb_predictions = np.zeros(len(df_test))
    for xgb_model in xgb_models:
        xgb_predictions += (xgb_model.predict(xgb.DMatrix(df_test.loc[:, xgb_config['features']])) / len(xgb_models))'''
        
    # PyTorch Dataset and DataLoader
    test_dataset = TabularDataset(features=df_test.loc[:, mlp_config['features']].values)
    test_loader = DataLoader(
        test_dataset,
        batch_size=len(test_dataset),
        sampler=SequentialSampler(test_dataset),
        pin_memory=True,
        drop_last=False,
        num_workers=0
    )
    
    # Multilayer Perceptron Inference
    mlp_predictions = np.zeros(len(df_test))
    with torch.no_grad():
        for features in test_loader:
            for model in mlp_models:
                outputs = model(features.to(torch.device('cuda')))
                mlp_predictions += (outputs.detach().cpu().numpy() / len(mlp_models))
    
    df_sample_submission['target'] = mlp_predictions
    
    env.predict(df_sample_submission)
